<a href="https://colab.research.google.com/github/rb58853/ML-RSI-Images/blob/main/Clip%20embbeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from PIL import Image
import torch

In [10]:
from transformers import CLIPProcessor, CLIPModel

device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [70]:
def get_image_embedding(image):
    image_process = processor(
       text=None,
       images=image,
       return_tensors="pt"
    )["pixel_values"].to(device)

    embedding = model.get_image_features(image_process)
    # print (embedding)
    # return embedding
    embedding_as_np = embedding.cpu().detach().numpy()
    return embedding
    # return embedding_as_np

In [100]:
def get_text_embedding(text):
    image =  Image.new('RGB', (10, 10), color = (0, 0, 0))
    encoded_text = processor(
        text = [text],
        images = image,
        padding=True,
        # truncation=True,
        # max_length=100,
        return_tensors='pt').to(device)

    outputs = model(**encoded_text)
    text_embeds = outputs['text_embeds']
    text_embedding_as_np = text_embeds.cpu().detach().numpy()
    return text_embeds
    # return text_embedding_as_np

    # pooler_output = outputs['text_model_output']['pooler_output']
    # return pooler_output

In [101]:
from scipy.spatial.distance import cosine

def cosine_similarity(vec1, vec2):
    vec1 = vec1.cpu().detach().numpy()
    vec2 = vec2.cpu().detach().numpy()
    vec1 = vec1[0]
    vec2 = vec2[0]
    return 1 - cosine(vec1, vec2)

In [137]:
image = Image.open('image_3.jpg')
# text = "a cartoon dog and a cat siting in a orange couch. Cartoon dog with a blue collar and brown nose. a gray and white cat with a smile on its face"
text = "a cartoon dog and a cat siting in a orange couch"
text = 'orange couch dog cat cartoon'

image_embedding = get_image_embedding(image)
text_embedding = get_text_embedding(text)

In [136]:
print(cosine_similarity(image_embedding, text_embedding))

0.3433797061443329


In [123]:
def normalize(embeddings):
   norms = embeddings.norm(p=2, dim=-1, keepdim=True)
   return embeddings / norms

def calculate_similarity(text_embedding, image_embedding):
   text_embedding = normalize(text_embedding)
   image_embedding = normalize(image_embedding)
   similarity = torch.matmul(text_embedding, image_embedding.t())
   return similarity


In [124]:
print(calculate_similarity(image_embedding, text_embedding))


tensor([[0.3434]], grad_fn=<MmBackward0>)
